In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Define all filters/kernels first
class ImageFilters:
    @staticmethod
    def get_sharpen_kernel():
        return np.array([[0, -1, 0],
                          [-1, 5, -1],
                          [0, -1, 0]], dtype=np.float32)
    
    @staticmethod
    def get_blur_kernel():
        return np.ones((3, 3), dtype=np.float32) / 9.0
    
    @staticmethod
    def get_sobel_x():
        return np.array([[-1, 0, 1],
                          [-2, 0, 2],
                          [-1, 0, 1]], dtype=np.float32)
    
    @staticmethod
    def get_sobel_y():
        return np.array([[-1, -2, -1],
                          [0,  0,  0],
                          [1,  2,  1]], dtype=np.float32)

# Custom convolution functions
class Convolution:
    @staticmethod
    def apply_manual_convolution(image, kernel):
        """Apply manual 2D convolution using vectorized operations"""
        kh, kw = kernel.shape
        pad_h, pad_w = kh // 2, kw // 2
        padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='reflect')
        windows = np.lib.stride_tricks.sliding_window_view(padded, (kh, kw))
        output = np.sum(windows * kernel, axis=(-2, -1))
        output = np.clip(output, 0, 255)
        return output.astype(np.uint8)
    
    @staticmethod
    def detect_corners(image):
        """Detect corners using cross Sobel derivatives"""
        gx = Convolution.apply_manual_convolution(image, ImageFilters.get_sobel_x())
        gy = Convolution.apply_manual_convolution(image, ImageFilters.get_sobel_y())
        response = np.abs(gx) * np.abs(gy)
        response = np.clip(response, 0, 255)
        return response.astype(np.uint8)
    
    @staticmethod
    def apply_opencv_convolution(image, kernel):
        """Apply OpenCV filter2D convolution"""
        return cv2.filter2D(image, -1, kernel)
    
    @staticmethod
    def apply_opencv_corner_detection(image):
        """Apply OpenCV Sobel for corner detection"""
        sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
        corners = np.abs(sobelx) * np.abs(sobely)
        return np.clip(corners, 0, 255).astype(np.uint8)

# Visualization functions
class Visualization:
    @staticmethod
    def display_comparison(images_dict):
        """Display images in a custom layout"""
        num_images = len(images_dict)
        rows = 3
        cols = (num_images + rows - 1) // rows  # Ceiling division
        
        plt.figure(figsize=(cols * 4, rows * 3))
        
        for i, (title, img) in enumerate(images_dict.items()):
            plt.subplot(rows, cols, i + 1)
            plt.imshow(img, cmap='gray')
            plt.title(title, fontsize=12)
            plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def create_interactive_window(image):
        """Create an interactive window with trackbars"""
        def update_dynamic_filter(val=None):
            try:
                center_value = cv2.getTrackbarPos('Center Weight', 'Dynamic Filter')
                edge_value = cv2.getTrackbarPos('Edge Weight', 'Dynamic Filter')
            except:
                # Default values if trackbars aren't ready
                center_value = 5
                edge_value = 1
            
            # Ensure values are at least 1
            center_value = max(1, center_value)
            edge_value = max(1, edge_value)
            
            # Build dynamic kernel
            dynamic_kernel = np.array([
                [0, -edge_value, 0],
                [-edge_value, center_value, -edge_value],
                [0, -edge_value, 0]
            ], dtype=np.float32)
            
            # Apply custom filter
            filtered_image = Convolution.apply_manual_convolution(image, dynamic_kernel)
            
            # Create display with kernel info
            kernel_info = f"Kernel:\n[0, -{edge_value}, 0]\n[-{edge_value}, {center_value}, -{edge_value}]\n[0, -{edge_value}, 0]"
            display_image = np.zeros((filtered_image.shape[0] + 100, filtered_image.shape[1]), dtype=np.uint8)
            display_image[100:, :] = filtered_image
            
            # Add kernel text
            for i, line in enumerate(kernel_info.split('\n')):
                cv2.putText(display_image, line, (10, 30 + i * 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            cv2.imshow('Dynamic Filter', display_image)
        
        # Create window and controls
        cv2.namedWindow('Dynamic Filter')
        cv2.waitKey(100)  # Ensure window is created
        cv2.createTrackbar('Center Weight', 'Dynamic Filter', 5, 10, update_dynamic_filter)
        cv2.createTrackbar('Edge Weight', 'Dynamic Filter', 1, 10, update_dynamic_filter)
        cv2.waitKey(100)  # Ensure trackbars are created
        
        # Initial update
        update_dynamic_filter()
        
        print("Press 'ESC' to close the interactive window")
        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == 27:  # ESC key
                break
        
        cv2.destroyAllWindows()

# Main execution
def main():
    # Load image
    try:
        original_image = cv2.imread('imagen1.jpg', cv2.IMREAD_GRAYSCALE)
        if original_image is None:
            raise FileNotFoundError("Image 'imagen1.jpg' not found in current directory")
    except Exception as e:
        print(f"Error loading image: {e}")
        return
    
    # Get filter kernels
    sharpen_kernel = ImageFilters.get_sharpen_kernel()
    blur_kernel = ImageFilters.get_blur_kernel()
    
    # Process with manual convolution
    manual_sharpen = Convolution.apply_manual_convolution(original_image, sharpen_kernel)
    manual_blur = Convolution.apply_manual_convolution(original_image, blur_kernel)
    manual_corners = Convolution.detect_corners(original_image)
    
    # Process with OpenCV
    opencv_sharpen = Convolution.apply_opencv_convolution(original_image, sharpen_kernel)
    opencv_blur = Convolution.apply_opencv_convolution(original_image, blur_kernel)
    opencv_corners = Convolution.apply_opencv_corner_detection(original_image)
    
    # Display results
    results = {
        'Original Image': original_image,
        'Manual Sharpen': manual_sharpen,
        'OpenCV Sharpen': opencv_sharpen,
        'Manual Blur': manual_blur,
        'OpenCV Blur': opencv_blur,
        'Manual Corner Detection': manual_corners,
        'OpenCV Corner Detection': opencv_corners
    }
    
    Visualization.display_comparison(results)
    
    # Show interactive window
    Visualization.create_interactive_window(original_image)

# Run the program
if __name__ == "__main__":
    main()